<a href="https://colab.research.google.com/github/lephuocdat2000/Advanced-CV/blob/main/Dog-and-Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
from PIL import Image
import numpy as np
import zipfile
from io import StringIO
import imghdr
import cv2 
from sklearn.model_selection import train_test_split
from torchvision import models
import torch
from torchvision import transforms
import tensorflow as tf

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(227),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [107]:
alexnet = models.alexnet(pretrained=True)
alexnet.classifier=alexnet.classifier[0:5]

#Đọc file data 

In [119]:
path_to_zip_file = '/content/drive/MyDrive/Thị giác máy tính/Thị giác máy tính nâng cao/Dog and Cat.zip'
imgzip = zipfile.ZipFile(path_to_zip_file)
inflist = imgzip.infolist()
images=[]
labels=[]
for f in inflist:
     if '.jpg' in f.filename:
       ifile = imgzip.open(f)
       img = Image.open(ifile)
       #Cho vào mạng alexnet
       img_t=transform(img)
       batch_t = torch.unsqueeze(img_t, 0)
       out = alexnet(batch_t)
      #  image_as_array = np.array(img, np.uint8)
      #  image_as_array = cv2.resize(image_as_array,(227,227))
       images.append(out)
       if 'dog' in f.filename: 
         labels.append(1)
       elif 'cat' in f.filename:
         labels.append(0)

Load pre-train model

In [113]:
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [151]:
X_train, X_test, y_train, y_test = train_test_split(images,labels, test_size=0.2, random_state=42)

In [148]:
from sklearn.svm import SVC
model = SVC(gamma='auto')
model.fit(np.array(X_train),y_train)

ValueError: ignored